<a href="https://colab.research.google.com/github/jscienciadados/ciencia-dados/blob/main/CBOW_Embargos_de_declacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Embargos de Declaração
#<font color="Navy">Processamento de Linguagem Natural</font>

# Estudo de Caso:
Integencia Artificial para Previsão de Sentenças em Embargos de Declaração

In [1]:
# Imports
import torch
import torch.nn as nn
import numpy as np

#<font color="SteelBlue">Preparando os Dados</font>
O texto abaixo é um exemplo de embargo de declaração. Embora o texto represente um embargo, dados críticos foram substituídos por informações genéricas, o que não compromete o objetivo do estudo de caso.

In [2]:
# Texto de embargo de declaração
embargo = """O embargante sofreu o ajuizamento de ação de danos morais e materiais, cujo objeto é o reaver os
valores pagos pelo sinal dado em um contrato de compra e venda de imóvel no qual não foi dado continuidade. Em 24
de fevereiro de 2012, o Magistrado proferiu decisão de fls. 277 a 280, que condenou todas as demandadas
solidariamente no seguinte teor: Diante de todo o exposto, com fundamento no art. 1234, I, do CPC/2015,
julgo procedentes em parte os pedidos constantes na inicial, condenando solidariamente as demandadas, XPTO LTDA,
BOB CAMARGO DE MORAES, a Pagarem título de indenização por danos morais, consoante fundamentação acima discorrida,
o montante de R$ 1.500,00 (um mil e quinhentos reais), corrigidos monetariamente pelo INPC desde a data
desta decisão, acrescidos de juros de 1% ao mês, a partir da citação; condeno ainda, à restituição do valor
pago pelo demandante como sinal da entrada do imóvel, descontando apenas 20% (vinte por cento), referente às
despesas, devendo incidir juros de 1% (um por cento) ao mês contados da citação e correção monetária pelo INPC a
partir da sentença. Contudo, data venia, houve omissão e obscuridade na referida decisão, haja vista que a omissão
se deu pela ausência dos julgamentos das preliminares (Necessidade de Perícia Técnica e a incompetência de
Juizado Especial) proposta posteriormente em aditamento de contestação (Fls 251 a 254) para impugnar áudios
juntados pelo embargado, autorizado a ser realizada pela Douta Magistrada em audiência de Conciliação,
instrução e julgamento de fls 235 e 236, por ausência de intimação anterior para realizar a já tratada
impugnação aos áudios anexados."""

In [3]:
# Limpeza do texto substituindo vírgulas e pontos por espaços e colocando as palavras em minúsculo
embargo = embargo.replace(',','').replace('.','').lower().split()


In [4]:
# Criação do corpus com o texto acima
corpus = set(embargo)

In [5]:
corpus

{'(fls',
 '(necessidade',
 '(um',
 '(vinte',
 '1%',
 '1234',
 '150000',
 '20%',
 '2012',
 '235',
 '236',
 '24',
 '251',
 '254)',
 '277',
 '280',
 'a',
 'acima',
 'acrescidos',
 'aditamento',
 'ainda',
 'ajuizamento',
 'anexados',
 'anterior',
 'ao',
 'aos',
 'apenas',
 'art',
 'as',
 'audiência',
 'ausência',
 'autorizado',
 'ação',
 'bob',
 'camargo',
 'cento)',
 'citação',
 'citação;',
 'com',
 'como',
 'compra',
 'conciliação',
 'condenando',
 'condeno',
 'condenou',
 'consoante',
 'constantes',
 'contados',
 'contestação',
 'continuidade',
 'contrato',
 'contudo',
 'correção',
 'corrigidos',
 'cpc/2015',
 'cujo',
 'da',
 'dado',
 'danos',
 'das',
 'data',
 'de',
 'decisão',
 'demandadas',
 'demandante',
 'descontando',
 'desde',
 'despesas',
 'desta',
 'deu',
 'devendo',
 'diante',
 'discorrida',
 'do',
 'dos',
 'douta',
 'e',
 'em',
 'embargado',
 'embargante',
 'entrada',
 'especial)',
 'exposto',
 'fevereiro',
 'fls',
 'foi',
 'fundamentação',
 'fundamento',
 'haja',
 'houve',
 

In [6]:
#length
corpus_length = len(corpus)

In [7]:
# Dicionario para TF-IDF
dic_palavra = {}
dic_inverso_palavra = {}

In [8]:
# Loop pelo corpus para criar os dicionários
for i, palavra in enumerate(corpus):
    dic_palavra[palavra] = i
    dic_inverso_palavra[i] = palavra


In [9]:
# lista para receber os dados
dados = []

In [10]:
# Loop pelo texto par extrair sentenças e palavras
for i in range(2, len(embargo) - 2):
    sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]
    target = embargo[i]
    dados.append((sentence, target))

In [11]:
# Visualiza os dados
print(dados[3])

(['o', 'ajuizamento', 'ação', 'de'], 'de')


**OBS** => As quatro palavras na lista serão os dados de entrada e a palavra fora, sera a variavel de saida.

#<font color="DarkSlateGray">Construção do Modelo CBoW</font>

In [12]:
# definindo o comprimento de cada embedding
embedding_length = 20

In [13]:
# Classe para o modelo
class CBoW(torch.nn.Module):

    # Método construtor
    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
       
        # Camada de entrada do modelo para criação da embedding
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        # Camadas lineares
        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
       
        # Camadas de ativação
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    # Passo (forward)
    def forward(self, inputs):
       
        # Aqui definimos a ordem ds camadas da rede neural
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    # Obtém a word_emdedding
    def get_word_emdedding(self, word):
        word = torch.LongTensor([dic_palavra[word]])
        return self.embeddings(word).view(1,-1)


In [14]:
# Cria o modelo CBoW
modelo = CBoW(corpus_length, embedding_length)

In [15]:
# Função de custo
loss_function = nn.NLLLoss()

In [16]:
# Otimizador do modelo (backpropagation)
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.01)


In [17]:
# Função para criar o vetor de sentenças, necessário para treinar o modelo
def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype = torch.long)


In [18]:
dic_palavra

{'(fls': 74,
 '(necessidade': 21,
 '(um': 150,
 '(vinte': 29,
 '1%': 43,
 '1234': 22,
 '150000': 142,
 '20%': 154,
 '2012': 130,
 '235': 10,
 '236': 96,
 '24': 3,
 '251': 132,
 '254)': 5,
 '277': 42,
 '280': 47,
 'a': 80,
 'acima': 127,
 'acrescidos': 107,
 'aditamento': 68,
 'ainda': 19,
 'ajuizamento': 167,
 'anexados': 40,
 'anterior': 51,
 'ao': 41,
 'aos': 104,
 'apenas': 111,
 'art': 15,
 'as': 137,
 'audiência': 9,
 'ausência': 136,
 'autorizado': 110,
 'ação': 75,
 'bob': 76,
 'camargo': 166,
 'cento)': 103,
 'citação': 109,
 'citação;': 48,
 'com': 148,
 'como': 82,
 'compra': 6,
 'conciliação': 140,
 'condenando': 131,
 'condeno': 113,
 'condenou': 60,
 'consoante': 72,
 'constantes': 153,
 'contados': 32,
 'contestação': 102,
 'continuidade': 1,
 'contrato': 120,
 'contudo': 37,
 'correção': 100,
 'corrigidos': 38,
 'cpc/2015': 73,
 'cujo': 44,
 'da': 152,
 'dado': 147,
 'danos': 16,
 'das': 97,
 'data': 175,
 'de': 83,
 'decisão': 117,
 'demandadas': 57,
 'demandante': 39,


In [19]:
# O dicionário de palavras será convertido em um vetor de sentenças. Aqui um exemplo:
print(make_sentence_vector(['pela','ausência','dos','julgamentos'], dic_palavra))


tensor([ 13, 136,  99,  64])


#<font color="DarkGreen">Treinamento do Modelo</font>

In [20]:
# Loop por 150 passadas (epochs) de treinamento
for epoch in range(150):
   
    # Inicia o erro da época com 0
    epoch_loss = 0
   
    # Loop pelos dados de entrada (sentence) e saída (target)
    for sentence, target in dados:
       
        # Inicializa os gradientes com zero
        modelo.zero_grad()
       
        # Cria o vetor de sentença com os dados de entrada (que devem estar no dicionário de palavras)
        sentence_vector = make_sentence_vector(sentence, dic_palavra)  
       
        # Usa o vetor para fazer previsões com o modelo e retorna as probabilidades
        log_probs = modelo(sentence_vector)
       
        # Calcula o erro do modelo
        loss = loss_function(log_probs, torch.tensor([dic_palavra[target]], dtype = torch.long))
       
        # Chama o método de backpropagation para calcular o gradiente da derivada
        loss.backward()
       
        # Otimiza os pesos do modelo e segue para a próxima passada
        # É aqui que o aprendizado acontece
        optimizer.step()
       
        # Atualiza o erro da época
        epoch_loss += loss.data
       
    # Imprime epoch e erro da epoch    
    print('Epoch: ' + str(epoch) + ', Erro do Modelo: ' + str(epoch_loss.item()))


Epoch: 0, Erro do Modelo: 1353.9764404296875
Epoch: 1, Erro do Modelo: 1228.389892578125
Epoch: 2, Erro do Modelo: 1135.167236328125
Epoch: 3, Erro do Modelo: 1046.96923828125
Epoch: 4, Erro do Modelo: 954.7318725585938
Epoch: 5, Erro do Modelo: 855.3926391601562
Epoch: 6, Erro do Modelo: 748.75048828125
Epoch: 7, Erro do Modelo: 638.1257934570312
Epoch: 8, Erro do Modelo: 527.9646606445312
Epoch: 9, Erro do Modelo: 424.32794189453125
Epoch: 10, Erro do Modelo: 332.28973388671875
Epoch: 11, Erro do Modelo: 253.50973510742188
Epoch: 12, Erro do Modelo: 189.61276245117188
Epoch: 13, Erro do Modelo: 139.90464782714844
Epoch: 14, Erro do Modelo: 103.4542465209961
Epoch: 15, Erro do Modelo: 77.75196075439453
Epoch: 16, Erro do Modelo: 60.30677795410156
Epoch: 17, Erro do Modelo: 47.98225784301758
Epoch: 18, Erro do Modelo: 39.18935775756836
Epoch: 19, Erro do Modelo: 32.67264938354492
Epoch: 20, Erro do Modelo: 27.775829315185547
Epoch: 21, Erro do Modelo: 24.081180572509766
Epoch: 22, Erro

#<font color="red">=></font> Observe como o erro foi reduzido a cada passada, nitidamente o aprendizado ocorrendo. Vamos agora usar o modelo para fazer previsões.

In [21]:
# Função para obter uma previsão
def get_resultado_previsto(input, dic_inverso_palavra):
    index = np.argmax(input)
    return dic_inverso_palavra[index]


In [22]:
# Função para prever sentenças (aplicamos aos novos dados o mesmo tratamento usado nos dados de treino)
def preve_sentenca(sentence):
   
    # Dividimos a sentença com split
    sentence_split = sentence.replace('.','').lower().split()
   
    # Criamos o vetor de sentença
    sentence_vector = make_sentence_vector(sentence_split, dic_palavra)
   
    # Faz a previsão com o modelo
    prediction_array = modelo(sentence_vector).data.numpy()
   
    # Print dos resultados
    print('Palavras Anteriores: {}\n'.format(sentence_split[:2]))
    print('Palavra Prevista: {}\n'.format(get_resultado_previsto(prediction_array[0], dic_inverso_palavra)))
    print('Palavras Seguintes: {}\n'.format(sentence_split[2:]))

#<font color="Chocolate">Previsões com o Modelo</font>
Dentro da frase: "ausência de intimação anterior para realizar", vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra intimação e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [23]:
# Previsão com o modelo
preve_sentenca('ausência de anterior para')


Palavras Anteriores: ['ausência', 'de']

Palavra Prevista: intimação

Palavras Seguintes: ['anterior', 'para']



In [24]:
# Emdedding da palavra
print(modelo.get_word_emdedding('intimação'))


tensor([[ 0.1381, -0.8328, -0.6479,  0.4492, -0.0958, -1.5677,  0.4467,  0.0566,
         -1.0356, -1.6364, -0.7501, -0.2022, -1.8816,  0.8985,  1.0868, -0.9119,
         -0.0781,  1.1288, -0.7877, -0.5345]], grad_fn=<ViewBackward>)


Perfeito! O modelo fez a previsão da sentença no Embargo de Declaração!  exemplo.

Dentro da frase: **"devendo incidir juros de 1%"**, vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra **juros** e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [25]:
# Previsão com o modelo
preve_sentenca('devendo incidir de 1%')


Palavras Anteriores: ['devendo', 'incidir']

Palavra Prevista: juros

Palavras Seguintes: ['de', '1%']



#<font color="red">Perfeito!</font>
O modelo fez a previsão da sentença no Embargo de Declaração.